In [1]:
from ipynb.fs.full.forcemodel import IForceModel
from ipynb.fs.full.integrator import IIntegrator
import numpy as np
import math

In [2]:
class SolarSystemEnv:
    def __init__(self, force_model: IForceModel, integrator: IIntegrator, bodies, dt=60.0):
        """
        bodies : array-like of masses, shape (N,)
        """
        self.dt = dt
        self.force_model = force_model()
        self.integrator = integrator()

        self.bodies = bodies
        self.n_bodies = len(self.bodies[0])
        dim = 2
        
        self.m = np.zeros((self.n_bodies), dtype=np.float64)  # positions
        self.x = np.zeros((self.n_bodies, dim), dtype=np.float64)  # positions
        self.v = np.zeros((self.n_bodies, dim), dtype=np.float64)  # velocities
        self.a = np.zeros((self.n_bodies, dim), dtype=np.float64)  # accelerations
        self.t = 0.0
        
        self.m, self.x, self.v, self.a, self.t = self.reset()

    def reset(self):
        self.m = self.bodies[0]
        self.x = self.bodies[1]
        self.v = self.bodies[2]
        self.a = self.force_model.acceleration(self.x, self.m)
        self.t = 0.0

        return self.m, self.x, self.v, self.a, self.t

    def step(self):
        self.t += self.dt
        self.x, self.v, self.a = self.integrator.step(
            self.m, self.x, self.v, self.a,
            self.force_model,
            self.dt
        )

        return self.x, self.v, self.a

In [3]:
def run_simulation(force_model: IForceModel, integrator: IIntegrator, bodies, dt=180.0, n_steps=100000, records_len=600):
    states = []
    step = max(1, math.floor(n_steps / records_len))

    env = SolarSystemEnv(force_model, integrator, bodies, dt=dt)
    env_state = env.reset()

    #ship = Spacecraft(dt=60.0)
    #ship.reset(env_state)
    ship_state = None


    for i in range(n_steps):
        x, v, a = env.step()
        #env_state = env.step()
        #ship.step(env_state)
        
        # record every nth step
        if (i % step == 0):
            #states.append(env_state.copy() )
            states.append({"m": env.m,"x": x,"v": v,"a": a,"t": env.t})

    return states

In [4]:
def adjust_barycentric(bodies):
    m = bodies[0].astype(np.float64).copy()
    x = bodies[1].astype(np.float64).copy()
    v = bodies[2].astype(np.float64).copy()
    n = bodies[3].astype(str).copy()
    c = bodies[4].astype(str).copy()
    
    M = np.sum(m)

    # Center of mass position
    R_cm = np.sum(m[:, None] * x, axis=0) / M
    # Shift all positions to barycentric frame
    x -= R_cm
    
    # Total momentum
    P = np.sum(m[:, None] * v, axis=0)
    # Adjust momentum of all bodies to 0 initial momentum
    V_cm = P / M
    v -= V_cm
    
    """
    # Ajust Suns velocity to cancel total momentum
    index_sun = 0
    V_corr = P / m[index_sun]
    P_corr = V_corr * m[index_sun]
    v[index_sun] -= V_corr
    print("Suns P to correctr: ", P_corr)
    print("Suns V to correctr: ", V_corr)

    sum = 0
    for i in range(len(m)):
        P_i = m[i] * v[i]
        print("P_i for ", i, ": ", P_i)
        sum += P_i
    print("P_sum: ", sum)
    """
    
    # Center of Mass
    print("COM position:",np.sum(m[:,None] * x, axis=0) / M)
    # Total Momentum
    print("Total momentum:", np.sum(m[:,None] * v, axis=0))

    barycentric_bodies = []
    barycentric_bodies.append(m)
    barycentric_bodies.append(x)
    barycentric_bodies.append(v)
    barycentric_bodies.append(n)
    barycentric_bodies.append(c)
    
    return barycentric_bodies